# SPECTRE-CPU-V2

# SETUP PRE-REQUISITES

In [ ]:
import os
import platform
import sys
import glob

import tensorflow as tf
#from tensorflow.keras import layers

#import keras
from tensorflow import keras
from keras import layers
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, Flatten, Input
from keras.models import Model
from keras import layers, models

import pandas as pd

from tabulate import tabulate

import seaborn as sns # Graphing, built ontop of MatPlot for ease-of-use and nicer diagrams.
import sklearn # For Machine Learning algorithms
import scikitplot # Confusion matrix plotting
from sklearn.decomposition import PCA # For PCA dimensionality reduction technique
from sklearn.preprocessing import StandardScaler # For scaling to unit scale, before PCA application
from sklearn.preprocessing import LabelBinarizer # For converting categorical data into numeric, for modeling stage
from sklearn.model_selection import StratifiedKFold # For optimal train_test splitting, for model input data
from sklearn.model_selection import train_test_split # For basic dataset splitting
from sklearn.neighbors import KNeighborsClassifier # K-Nearest Neighbors ML classifier (default n. of neighbors = 5)
from scikitplot.metrics import plot_confusion_matrix # For plotting confusion matrices
from sklearn.metrics import accuracy_score # For getting the accuracy of a model's predictions
from sklearn.metrics import classification_report # Various metrics for model performance
from sklearn.neural_network import MLPClassifier # For Neural Network classifier
from sklearn.linear_model import LogisticRegression

def escape():
    sys.exit()

In [ ]:
print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print()
print(f"Python {sys.version}")

## ENVIRONMENT SETUP

**Setup INFO level**

In [ ]:
tf.get_logger().setLevel('INFO')

#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

**Useful environment variables**

In [ ]:
# 'Reduced dimensions' variable for altering the number of PCA principal components. Can be altered for needs.
# Only 7 principal components needed when using non-normalised PCA dataset.
dimensions_num_for_PCA = 7

# Max number of permutations to run. Can be altered for needs.
number_of_permutations = 100

# 10 folds is usually the heuristic to follow for larger datasets of around this size.
num_of_splits_for_skf = 10

# Seed value to pass into models so that repeated runs result in the same output
seed_val = 1

# Number of statistical distance measures to run (for the results, columns section)
num_of_statistical_dist_measures = 6

## METHODS

`Clean_dataset()` method is used to remove infinite and Nan value errors (in the original dataset), which can cause issues in the PCA transform step.
- `assert` keyword can be used for testing purposes. If the input param is not of type Pandas Dataframe, then the error message will be shown. Thus, you 'assert' that a pd.Dataframe is input.
- `.dropna(inplace=True)` drops any rows which contain NaN/ Null values. NaN values would cause issues later on, with model training and other functions. A lot of functions require NaN values to be removed to work (see docs: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html).

Code reference: https://stackoverflow.com/a/46581125 (with a minor change = removed the conversion to float64 type)

In [ ]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep]

`get_PCA_feature_names() `method is used to generate feature names for the number of PCA components passed in as a param. Returns a list of feature names for principal component column headings, in a Pandas Dataframe.
- No special code here, just pure Python code. Instantiating a list, and populating it with strings for the PCA header names. Then, appending each to the list and returning it.

In [ ]:
def get_PCA_feature_names(num_of_pca_components):
    feature_names = []
    for i in range(num_of_pca_components):    
        feature_names.append(f"Principal component {i+1}")
    return feature_names

# ANN TRAINING

## Dataset Import

In [ ]:
Friday_Morning_Data = pd.read_csv('../../dataset/CICIDS2017/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
df = Friday_Morning_Data.copy()
df.head()

## Data Prepocessing

**Fixing column name issues**

Because of Excel being used to create the csv, the column headings/ names contain whitespace padding, incorrect capitalisation, etc... which makes it difficult to correctly select by column names. This piece of code below just removes these issues.

- df_columns can be used to access all the column heading names (see docs: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.columns.html)
- The code below acts on each of the header names (for ease of use):
  - str.strip() just strips/ removes any leading and trailing spaces
  - str.lower() just converts all characters to lower case
  - str.replace('x', 'y') just replaces all instances of x with y, i.e. changing spaces to '_'
  - str.replace('x', '') can be used to remove the specified x characters (replacing with nothing/ empty char)
Code Reference: https://medium.com/@chaimgluck1/working-with-pandas-fixing-messy-column-names-42a54a6659cd

In [ ]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df.head()

**Datatset Data Types**

In [ ]:
df.dtypes

**Fixing issues with ScikitLearn's PCA transform on this dataset**

Without cleaning the dataset, the PCA transform will throw this error:
- "sklearn error ValueError: Input contains NaN, infinity or a value too large for dtype('float64')".

In [ ]:
df_cleaned = df.copy()
df_cleaned = clean_dataset(df_cleaned) # see methods at top of notebook
df_cleaned

**Resetting indexes since rows have been dropped. See the difference between above dataframe indexes and below.**
- `.reset_index()` method resets the Pandas Dataframe indexes, for the rows. Useful to do after removing rows, as this messes up the indexes. It creates a new 'index' column that needs to be dropped, as it's useless (see docs: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html)
- `.drop([column_name], axis=x, inplace=y)` drops a specified column from the dataframe and returns a new copy. When `inplace=True`, it transforms the Dataframe itself (instead of needing to copy). The `axis` specifies the dimension of what to drop i.e. if **axis=0, it drops a row. If axis=1, it drops a column**.

In [ ]:
df_cleaned = df_cleaned.reset_index()
# Removing un-needed index column added by reset_index method
df_cleaned.drop('index', axis=1, inplace=True)
df_cleaned

**What features should be included from PCA, and why?**

Looking at the list of feature names in the dataset (shown below), one can see that all other features should be of numeric type (with domain knowledge). They're all currently numeric type (either float or int). Consequently, PCA can be fully applied.
- `df.columns.tolist()` converts the Dataframe column names into a Python list

In [ ]:
df.columns.tolist()

**Data Preparation: PCA Dimension reduction and scaling (Hughes' Phenomenon)**

PCA acts to reduce the dimensions/ search space of the dataset as much as possible, while trying to maintain the most information possible e.g. It can easily reduce the dimensionality by more than half, while still maintaining 99% of the original data's information- it does this by extracting out the most important information/ trends/ spread (variance) of each dimension/ attribute- into n 'principal components'.

More formally: PCA is used to decompose a multivariate dataset in a set of successive orthogonal components that explain a maximum amount of the variance.

>Key note:
> > "PCA centers but does not scale the input data for each feature before applying the SVD. The optional parameter whiten=True makes it possible to project the data onto the singular space while scaling each component to unit variance. This is often useful if the models down-stream make strong assumptions on the isotropy of the signal: this is for example the case for Support Vector Machines with the RBF kernel and the K-Means clustering algorithm." (https://scikit-learn.org/stable/modules/decomposition.html#pca)

PCA still works without standardizing the features to unit scale but tranforming to unit scale should still be done to prevent large variance features from having an over-bearing affect on other lower variance features (via something like StandardScaler here https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html).

This is particularly important with this dataset, as some features have massively wide variances and others do not (e.g. the 'idle_std' values can range from e+06, all the way to zero).

- `Dataframe['x']` syntax allows the selection of one of the Dataframe columns (where 'x' is the column heading/ name).
- `df_Series.unique()` returns an array, showing all the unique values inside the Pandas Series object.

In [ ]:
# Saving the label attribute before dropping it.
df_labels = df_cleaned['label']
# Shows all the possible labels/ classes a model can predict.
# Need to alter these to numeric 0, 1, etc... for model comprehension (e.g. pd.get_dummies()).
df_labels.unique()

The label column has to be removed as you don't want this involved in the PCA process. It can be concatted back with the PCA tranformed dataframe.

In [ ]:
# Axis=1 means columns. Axis=0 means rows. inplace=False means that the original 'df' isn't altered.
df_no_labels = df_cleaned.drop('label', axis=1, inplace=False)
# Getting feature names for the StandardScaler process
df_features = df_no_labels.columns.tolist()
# Printing out Dataframe with no label column, to show successful dropping
df_no_labels

**Using StandardScaler to transform features into unit scale (optional for PCA)**

- `StandardScaler()` is an imported model from the sklearn.preprocessing library. It scales the specified Pandas Dataframe or Series object values to unit scale/ variance. This is usually required for certain functions to perform correctly, e.g. the PCA transform later (see docs: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
- `StandardScaler().fit_transform(df)` fits the StandardScaler model to the data, and transforms it into unit scale.
- `pd.Dataframe(data=x, columns=y)` can convert the data 'x' into a Pandas Dataframe object, using the respective columns 'y'

Code references: https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60

In [ ]:
df_scaled = StandardScaler().fit_transform(df_no_labels)
# Converting back to dataframe
df_scaled = pd.DataFrame(data = df_scaled, columns = df_features)
df_scaled

**Fitting and transforming the data with PCA**

Thus, the optimal number of principle components is set to the environment variable and this is now used to produce the appropriate multi-dimensional principle component array. This will be formatted back to a Pandas dataframe afterwards.

References: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html and https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60

In [ ]:
pca = PCA(n_components=dimensions_num_for_PCA)
#principal_components = pca.fit(df_scaled).transform(df_scaled) => for normalised PCA

# Non-normalised PCA
principal_components = pca.fit(df_no_labels).transform(df_no_labels)
principal_components

**Getting Principal Component feature names, dynamically**

Getting the Principal Component feature names, dynamically, for the optimal number of components (passed in as a param). Allows dynamic changing of PCs used.

> i.e. One can change the 'dimensions_num_for_PCA' environment variable, and all the code will still work.

In [ ]:
# See Methods at the top of the notebook
principal_component_headings = get_PCA_feature_names(dimensions_num_for_PCA)

Turning the Principal Components back into a Pandas Dataframe, ready for concatting back with the label feature.

In [ ]:
df_pc = pd.DataFrame(data = principal_components, columns = principal_component_headings)
df_pc

**Joining/ concatinating the label feature back onto the pca transformed dataset.**

Label still needs to be transformed into binary data (for model comprehension/ understanding i.e. the model doesn't understand string data but string data can be transformed into numeric data, which is model can understand and use).

See the Pandas.concat docs here: https://pandas.pydata.org/docs/reference/api/pandas.concat.html

In [ ]:
df_final = pd.concat([df_pc, df_labels], axis = 1)
# Scroll to the RHS end of dataframe to see attached label feature
df_final

**Transforming the label feature's categorical data into numeric data (via LabelBinarizer)**

Again, a model can't understand, for example, 'yes' and 'no' strings but... these can be mapped to a 1 for yes and a 0 for no. Then a model can understand, as it requires numeric input to distinguish the feature's domain of values.

The `sklearn.preprocessing.LabelBinarize`r can be used to convert the column data into binary numbers, which will then be correctly interpreted.

- Fit the List- this tells the LabelBinarizer what values exist, and how to map them.
- Call transform, passing a List, and this will return the encoded List.

> (Note: if label column has more than 2 unique labels, pandas.get_dummies is required instead)

(Code reference: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html)

In [ ]:
lb = LabelBinarizer()
df_final['label'] = lb.fit_transform(df_final['label'])
df_final

Showing the transformation. Again, to note, if label isn't binary then pd.get_dummies is required.

In [ ]:
print("Before LabelBinarizer: ", df_labels.unique())
print("After LabelBinarizer: ", df_final['label'].unique())

In [ ]:
df_final

## Training

### Dataset Splitting

**K-Fold Cross Validation and Stratified splitting**

K-Fold is a technique which splits data into K folds (splits). Train of a model K times, and for each training iteration, K-Fold selects a different fold to use for testing; the remaining K - 1 folds become the training data. Typically, the optimal K value can be derived using the size of your dataset (num of rows). Ideally, each fold should be statistically representative of the population. Too small and it won't be useful. Too large, and you lose the positives from doing K-Fold.

You can use Stratified splitting with K-Fold, which ensures balance between some criteria (balances out the classes) e.g. equal portion of label classes in each fold.

Class Imbalance is a significant issue in the ML/ Data Mining domain. It leads to incorrect results e.g. if one fold had all of 1 label (accidentally), then it would produce terrible predictive results as it wouldn't know what the other label class data point would look like. You can only work with the data you have, so this has to be dealt with.

Benefits of K-Fold:
- Use more of the data towards making a succesful model.
- Obtain K models to evaluate, can improve the confidence that you have selected an appropriate model algorithm and cleaned/ prepared the data correctly, e.g. normal split with 1 model, one doesn't know if it's good or not- it could be heavily biased. Multiple models ensures less bias and increased variance.
- Looking at the accuracy results from each of the k-Folds, you can identify data issues e.g. a certain fold performs really badly. Could this suggest that more cleaning is required? Maybe the data preparation was performed incorrectly?
- If all folds return similar accuracies, one can be more confident that a deployed model will perform similarly to how one expects.

Issues with K-Fold:
- Creating K separate models requires more computation.
- If you haven't got much data, you might not get many folds. Less folds means K-Fold loses its benefits.
- If K is very large, each fold is small, and harder to ensure statistical distribution of.
- Choosing the best of K models introduces bias. Real world data could perform better under a more general, lower performing model.

Code reference: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html

In [ ]:
# Separating the label so that the answers aren't provided to the model, in training.
X = df_final.drop(['label'], axis = 1)
y = df_final['label']
y

In [ ]:
skf = StratifiedKFold(n_splits=num_of_splits_for_skf, shuffle=False)
skf

Now, splitting the data into train and test data, using the optimal splitting techniques of K-Fold and Stratified Splitting.

In [ ]:
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    reshaped_y_train = np.asarray(y_train).reshape(-1, 1)
    reshaped_y_test = np.asarray(y_test).reshape(-1, 1)
    
print( 'X_train length: ', len(X_train) ) # To check if splits worked
print( 'y_train length: ', len(y_train) )
print( 'X_test length: ', len(X_test) )
print( 'y_test length: ', len(y_test) )

### Modeling

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from keras.regularizers import l1, l2, l1_l2

# Define the ANN model

#model = Sequential([
#    Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)),
#    BatchNormalization(),
#    Dropout(0.5),
#    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
#    BatchNormalization(),
#    Dropout(0.5),
#    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
#    BatchNormalization(),
#    Dropout(0.5),
#    Dense(1, activation='sigmoid')
#])


#model = Sequential([
#    Dense(256, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)),
#    BatchNormalization(),
#    Dropout(0.5),
#    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
#    BatchNormalization(),
#    Dropout(0.5),
#    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
#    BatchNormalization(),
#    Dropout(0.5),
#    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
#    BatchNormalization(),
#    Dropout(0.5),
#    Dense(1, activation='sigmoid')
#])

#model = Sequential([
#    Dense(512, kernel_initializer='he_normal', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)),
#    LeakyReLU(alpha=0.1),
#    BatchNormalization(),
#    Dropout(0.4),
#    Dense(256, kernel_initializer='he_normal', kernel_regularizer=l2(0.001)),
#    LeakyReLU(alpha=0.1),
#    BatchNormalization(),
#    Dropout(0.4),
#    Dense(128, kernel_initializer='he_normal', kernel_regularizer=l2(0.001)),
#    LeakyReLU(alpha=0.1),
#    BatchNormalization(),
#    Dropout(0.4),
#    Dense(64, kernel_initializer='he_normal', kernel_regularizer=l2(0.001)),
#    LeakyReLU(alpha=0.1),
#    BatchNormalization(),
#    Dropout(0.4),
#    Dense(32, kernel_initializer='he_normal', kernel_regularizer=l2(0.001)),
#    LeakyReLU(alpha=0.1),
#    BatchNormalization(),
#    Dropout(0.4),
#    Dense(1, activation='sigmoid')
#])

model = Sequential([
    Dense(256, kernel_initializer='he_normal', input_shape=(X_train.shape[1],), kernel_regularizer=l1_l2(l1=0.0001, l2=0.0001)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.2),
    Dense(128, kernel_initializer='he_normal', kernel_regularizer=l1_l2(l1=0.0001, l2=0.0001)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, kernel_initializer='he_normal', kernel_regularizer=l1_l2(l1=0.0001, l2=0.0001)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, kernel_initializer='he_normal', kernel_regularizer=l1_l2(l1=0.0001, l2=0.0001)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])


In [ ]:
model.summary()

In [ ]:
# Compile the model
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.RMSprop(), metrics=['accuracy'])

In [ ]:
# Train the model
#model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.2)
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test set accuracy: {:.2f}".format(accuracy))

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# Make predictions
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Calculate performance metrics
conf_matrix = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-score: {:.2f}".format(f1))

# EXPORT MODEL

In [ ]:
# Export as SavedModel
tf.saved_model.save(model, '/home/aryn/spectre-dev/spectre-code/spectre-ann/Model/DDOS_2/SavedModel/')

# Export as Keras Model
model.save("/home/aryn/spectre-dev/spectre-code/spectre-ann/Model/DDOS_2/spectre_ddos_2_hd5")

# Export as Keras H5 Model
model.save("/home/aryn/spectre-dev/spectre-code/spectre-ann/Model/DDOS_2/spectre_ddos_2_h5.h5")

# ANN ACTIVATION GRAPH